# Descomposición del DOM

In [ ]:
with open(r"www_youtube_com.txt","r",encoding="utf-8") as h:
    texto_html = h.read()

# Motor de Inferencia

In [ ]:
from dotenv import load_dotenv
import google.generativeai as genai
import zipfile
import os
import re

In [ ]:
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.5-flash")